### stage_model_classifier

prediction models of sign classifiers on stage dataset

data format:
    target , eventid ,    car_number,    stageid,     features...

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# to use only one GPU.
# use this on r-001
# otherwise comment
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb


In [3]:
# bulid regression model
classifiers = ['currank','avgrank','dice','lr','lrl1','lsvc','lsvcl2','rf','lrbias','xgb']
def get_classifier(classifier = 'lr'):
    
    class_weight = None
    
    if classifier == "lsvc":
        clf = LinearSVC(penalty='l1',dual=False, tol=1e-3, class_weight=class_weight )
    elif classifier == "lsvcl2":
        clf = LinearSVC(penalty='l2', tol=1e-4, class_weight=class_weight)
    elif classifier == 'rf':
        #clf = RandomForestClassifier(n_estimators=100, n_jobs=4,criterion='entropy', min_samples_split=1,class_weight = class_weight)
        clf = RandomForestClassifier(n_estimators=100, n_jobs=-1,criterion='entropy', class_weight = class_weight)
    elif classifier == 'lr':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = False, verbose = 0)
    elif classifier == 'lrbias':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = True, verbose = 1)
    elif classifier == 'lrl1':
        clf = LogisticRegression(class_weight = class_weight, penalty='l1',n_jobs=-1)
    elif classifier == 'xgb':
        clf = xgb.XGBClassifier(booster = 'gbtree', nthread = -1, subsample = 1, 
                                n_estimators = 600, colsample_bytree = 1, max_depth = 6, min_child_weight = 1)
    elif classifier == 'dice':
        clf = RandomDice('1234')
    elif classifier == 'currank':
        clf = CurRank()
    elif classifier == 'avgrank':
        clf = AverageRank()        
    else:
        clf = None
        
    return clf


class CurRank():
    """
    predict with current rank
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = [0 for x in range(test_x.shape[0])]
        return np.array(pred_y)
    
class AverageRank():
    """
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            #13, change_in_rank_all
            pred_y.append(x[13])
        pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in pred_y])
        return np.array(pred_y_avg)   

class RandomDice():
    """
    a random dice model
    """
    def __init__(self, seed='1234'):
        self.dist = []
        self.val = []
        random.seed(seed)
    
    def fit(self, x, y):
        total = y.shape[0]
        yval = set(y)
        
        ratio = 0.
        for val in yval:
            self.val.append(val)
            ratio += np.sum(y==val)*1.0 / total
            self.dist.append(ratio)
            
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            dice = random.random()
            #search in self.dist
            find_idx = -1
            for idx, ratio in enumerate(self.dist):
                if dice <= ratio:
                    find_idx = idx
                    break
            
            #or the last one match
            pred_y.append(self.val[find_idx])
            
        return np.array(pred_y)

def evaluate(test_y, pred_y):
    precision = metrics.precision_score(test_y, pred_y, average=None) 
    recall = metrics.recall_score(test_y, pred_y, average=None)
    f1 = metrics.f1_score(test_y, pred_y, average=None)
    accuracy = metrics.accuracy_score(test_y, pred_y)
    print('precision=%s, recall=%s, f1=%s, accuracy=%.2f'%(precision,recall, f1, accuracy))
    return accuracy
    
#
#features
#    cols=[Myidx, 'target','eventid','car_number','stageid',
#             'firststage','pit_in_caution','start_position',
#             'start_rank','start_rank_ratio','top_pack','bottom_pack',
#             'average_rank','average_rank_all',
#             'change_in_rank','change_in_rank_all','rate_of_change','rate_of_change_all']    
def split_by_eventid(stagedata, eventid):
    """
    split by eventid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['eventid'] != eventid].to_numpy()
    test  = stagedata[stagedata['eventid'] == eventid].to_numpy()

    #2:car_number
    train_x = train[:,2:]
    train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    test_x = test[:,2:]
    test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    
    return train, test, train_x, train_y, test_x, test_y


def split_by_stageid(stagedata, stageid):
    """
    split by stageid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['stageid'] <= stageid].to_numpy()
    test  = stagedata[stagedata['stageid'] > stageid].to_numpy()

    train_x = train[:,2:]
    train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    test_x = test[:,2:]
    test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    
    return train, test, train_x, train_y, test_x, test_y


### baseline
def baseline_model():
    #1. predict with current rank, rankchg = 0
    print('[*] predict with current rank, rankchg = 0')
    pred_y_simple = np.zeros_like(test_y)
    score1 = evaluate(test_y, pred_y_simple)

    #2. predict with average rankchg (change_in_rank_all):idx = 15
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    score2 = evaluate(test_y, pred_y_avg)
    return score1, score2

def classifier_model(name='lr'):
    ### test learning models
    print('[*] predict with %s model'%name)
    clf = get_classifier(name)
    clf.fit(train_x, train_y)

    pred_y = clf.predict(test_x)
    score = evaluate(test_y, pred_y)
    return score

In [4]:
#load data
suffix='-withneighbor-newfeatures-timediff'
stagedata = pd.read_csv('stage-2018%s.csv'%suffix)
stagedata.fillna(0, inplace=True)
stagedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 805 entries, 0 to 804
Data columns (total 35 columns):
Unnamed: 0                   805 non-null int64
target                       805 non-null int64
eventid                      805 non-null int64
car_number                   805 non-null int64
stageid                      805 non-null int64
firststage                   805 non-null int64
pit_in_caution               805 non-null int64
start_position               805 non-null int64
start_rank                   805 non-null int64
start_rank_ratio             805 non-null float64
top_pack                     805 non-null int64
bottom_pack                  805 non-null int64
average_rank                 805 non-null float64
average_rank_all             805 non-null float64
change_in_rank               805 non-null int64
change_in_rank_all           805 non-null float64
rate_of_change               805 non-null int64
rate_of_change_all           805 non-null float64
laptime_green_mean_pr

In [5]:
stagedata.head(5)

,Unnamed: 0,target,eventid,car_number,stageid,firststage,pit_in_caution,start_position,start_rank,start_rank_ratio,...,laptime_std_all,laps_prev,laps_after_last_pitstop,pittime_prev,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank
0,0,0,0,1,0,0,0,7,7,0.304348,...,0.000000,0,0,0.00000,0,0,0,0,0,0
1,1,-2,0,1,1,1,1,7,7,0.304348,...,6.431794,43,43,11.54325,-2,-1,1,3,0,-1
2,2,6,0,1,2,1,0,7,5,0.217391,...,7.771622,76,76,59.63585,0,-2,7,-3,18,2
3,3,-7,0,1,3,1,0,7,11,0.478261,...,6.740054,57,57,40.43850,-12,-9,-18,-6,-7,4
4,4,-3,0,1,4,1,1,7,4,0.173913,...,6.458577,56,56,39.51240,-8,-2,-2,-4,2,-3


In [6]:
cols = ['runid','trainsize','testsize','testdistribution']
cols.extend(classifiers)
print('cols:%s'%cols)
retdf = pd.DataFrame([],columns=cols)

eventsname = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']
events = set(stagedata['eventid'])
for eventid in events:
    print('Testset = %s'%eventsname[eventid])
    
    train, test, train_x, train_y, test_x, test_y = split_by_eventid(stagedata, eventid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
    #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
    #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))
    
    #record
    rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]
    
    acc = [0 for x in range(len(classifiers))]
    for idx, clf in enumerate(classifiers):
        acc[idx] = classifier_model(clf)

    rec.extend(acc)
    print('rec:%s'%rec)
    
    #new df
    df = pd.DataFrame([rec],columns=cols)
    retdf = pd.concat([retdf, df])        
    
retdf.to_csv('crossvalid_stagedata_splitbyevent%s.csv'%suffix)
df_event = retdf

cols:['runid', 'trainsize', 'testsize', 'testdistribution', 'currank', 'avgrank', 'dice', 'lr', 'lrl1', 'lsvc', 'lsvcl2', 'rf', 'lrbias', 'xgb']
Testset = Phoenix
[*] predict with currank model
precision=[0.        0.1754386 0.       ], recall=[0. 1. 0.], f1=[0.         0.29850746 0.        ], accuracy=0.18
[*] predict with avgrank model
precision=[0.3877551  0.22580645 0.17647059], recall=[0.34545455 0.35       0.15384615], f1=[0.36538462 0.2745098  0.16438356], accuracy=0.28
[*] predict with dice model
precision=[0.44897959 0.0625     0.36734694], recall=[0.4        0.05       0.46153846], f1=[0.42307692 0.05555556 0.40909091], accuracy=0.36
[*] predict with lr model
precision=[0.54545455 0.36363636 0.61538462], recall=[0.76363636 0.2        0.41025641], f1=[0.63636364 0.25806452 0.49230769], accuracy=0.54
[*] predict with lrl1 model


/scratch/hpda/anaconda3/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/py2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/py2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/scratch/

precision=[0.55555556 0.42857143 0.52272727], recall=[0.63636364 0.15       0.58974359], f1=[0.59322034 0.22222222 0.55421687], accuracy=0.54
[*] predict with lsvc model
precision=[0.58333333 0.46153846 0.49056604], recall=[0.50909091 0.3        0.66666667], f1=[0.54368932 0.36363636 0.56521739], accuracy=0.53
[*] predict with lsvcl2 model


/scratch/hpda/anaconda3/envs/py2/lib/python2.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


precision=[0.54545455 0.         0.45945946], recall=[0.76363636 0.         0.43589744], f1=[0.63636364 0.         0.44736842], accuracy=0.52
[*] predict with rf model
precision=[0.56451613 0.5        0.54545455], recall=[0.63636364 0.2        0.61538462], f1=[0.5982906  0.28571429 0.57831325], accuracy=0.55
[*] predict with lrbias model
[LibLinear]precision=[0.55737705 0.42857143 0.52173913], recall=[0.61818182 0.15       0.61538462], f1=[0.5862069  0.22222222 0.56470588], accuracy=0.54
[*] predict with xgb model
precision=[0.55555556 0.3        0.48780488], recall=[0.63636364 0.15       0.51282051], f1=[0.59322034 0.2        0.5       ], accuracy=0.51
rec:['Phoenix', 691, 114, '+:39,0:20,-:55', 0.17543859649122806, 0.2807017543859649, 0.35964912280701755, 0.543859649122807, 0.5350877192982456, 0.5263157894736842, 0.5175438596491229, 0.5526315789473685, 0.5350877192982456, 0.5087719298245614]
Testset = Indy500
[*] predict with currank model
precision=[0.         0.10222222 0.        ]

precision=[0.4375     0.27272727 0.45098039], recall=[0.57142857 0.07692308 0.60526316], f1=[0.49557522 0.12       0.51685393], accuracy=0.43
rec:['Pocono', 679, 126, '+:38,0:39,-:49', 0.30952380952380953, 0.30952380952380953, 0.3888888888888889, 0.4603174603174603, 0.40476190476190477, 0.38095238095238093, 0.3888888888888889, 0.42857142857142855, 0.42063492063492064, 0.42857142857142855]
Testset = Gateway
[*] predict with currank model
precision=[0.         0.25961538 0.        ], recall=[0. 1. 0.], f1=[0.         0.41221374 0.        ], accuracy=0.26
[*] predict with avgrank model
precision=[0.38095238 0.22580645 0.25806452], recall=[0.35555556 0.25925926 0.25      ], f1=[0.36781609 0.24137931 0.25396825], accuracy=0.30
[*] predict with dice model
precision=[0.48888889 0.3125     0.39534884], recall=[0.48888889 0.18518519 0.53125   ], f1=[0.48888889 0.23255814 0.45333333], accuracy=0.42
[*] predict with lr model
precision=[0.51470588 0.57142857 0.48275862], recall=[0.77777778 0.14814

In [7]:
df_event

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lrl1,lsvc,lsvcl2,rf,lrbias,xgb
0,Phoenix,691,114,"+:39,0:20,-:55",0.175439,0.280702,0.359649,0.543860,0.535088,0.526316,0.517544,0.552632,0.535088,0.508772
0,Indy500,580,225,"+:88,0:23,-:114",0.102222,0.248889,0.346667,0.515556,0.520000,0.524444,0.488889,0.382222,0.511111,0.400000
0,Texas,678,127,"+:35,0:30,-:62",0.236220,0.275591,0.354331,0.488189,0.472441,0.425197,0.503937,0.456693,0.496063,0.377953
0,Iowa,696,109,"+:39,0:25,-:45",0.229358,0.293578,0.394495,0.596330,0.596330,0.596330,0.403670,0.504587,0.596330,0.449541
0,Pocono,679,126,"+:38,0:39,-:49",0.309524,0.309524,0.388889,0.460317,0.404762,0.380952,0.388889,0.428571,0.420635,0.428571
0,Gateway,701,104,"+:32,0:27,-:45",0.259615,0.298077,0.423077,0.509615,0.500000,0.500000,0.480769,0.538462,0.509615,0.538462


In [8]:
retdf = pd.DataFrame([],columns=cols)

for stageid in range(8):
    train, test, train_x, train_y, test_x, test_y =split_by_stageid(stagedata, stageid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
    #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
    #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))
    
    #record
    rec = ['stage%d'%stageid,train_x.shape[0],test_x.shape[0],test_distribution]
    
    acc = [0 for x in range(len(classifiers))]
    for idx, clf in enumerate(classifiers):
        acc[idx] = classifier_model(clf)

    rec.extend(acc)
    print('rec:%s'%rec)
    
    #new df
    df = pd.DataFrame([rec],columns=cols)
    retdf = pd.concat([retdf, df])  
    
retdf.to_csv('crossvalid_stagedata_splitbystage%s.csv'%suffix)
df_stage = retdf

[*] predict with currank model
precision=[0.         0.21472393 0.        ], recall=[0. 1. 0.], f1=[0.         0.35353535 0.        ], accuracy=0.21
[*] predict with avgrank model
precision=[0.3796034 0.2244898 0.224    ], recall=[0.44816054 0.07857143 0.2629108 ], f1=[0.41104294 0.11640212 0.24190065], accuracy=0.31
[*] predict with dice model
precision=[0.45263158 0.23529412 0.33584906], recall=[0.43143813 0.17142857 0.41784038], f1=[0.44178082 0.19834711 0.37238494], accuracy=0.37
[*] predict with lr model
precision=[0.51450677 0.         0.55555556], recall=[0.88963211 0.         0.35211268], f1=[0.65196078 0.         0.43103448], accuracy=0.52
[*] predict with lrl1 model
precision=[0.49886621 0.         0.46445498], recall=[0.73578595 0.         0.4600939 ], f1=[0.59459459 0.         0.46226415], accuracy=0.49
[*] predict with lsvc model
precision=[0.5155642  0.         0.55797101], recall=[0.88628763 0.         0.36150235], f1=[0.65190652 0.         0.43874644], accuracy=0.52
[*]

precision=[0.57446809 0.45762712 0.30952381], recall=[0.41538462 0.51923077 0.41935484], f1=[0.48214286 0.48648649 0.35616438], accuracy=0.45
[*] predict with lsvc model
precision=[0.57142857 0.48837209 0.28571429], recall=[0.49230769 0.40384615 0.4516129 ], f1=[0.52892562 0.44210526 0.35      ], accuracy=0.45
[*] predict with lsvcl2 model
precision=[0.18181818 0.35294118 0.225     ], recall=[0.03076923 0.11538462 0.87096774], f1=[0.05263158 0.17391304 0.35761589], accuracy=0.24
[*] predict with rf model
precision=[0.53658537 0.57894737 0.29787234], recall=[0.67692308 0.21153846 0.4516129 ], f1=[0.59863946 0.30985915 0.35897436], accuracy=0.47
[*] predict with lrbias model
[LibLinear]precision=[0.58536585 0.4375     0.3255814 ], recall=[0.36923077 0.53846154 0.4516129 ], f1=[0.45283019 0.48275862 0.37837838], accuracy=0.45
[*] predict with xgb model
precision=[0.59615385 0.44680851 0.32653061], recall=[0.47692308 0.40384615 0.51612903], f1=[0.52991453 0.42424242 0.4       ], accuracy=0

In [9]:
#xgb max_tree_depth=3
df_stage

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lrl1,lsvc,lsvcl2,rf,lrbias,xgb
0,stage0,153,652,"+:213,0:140,-:299",0.214724,0.308282,0.371166,0.523006,0.487730,0.524540,0.521472,0.429448,0.510736,0.389571
0,stage1,288,517,"+:164,0:123,-:230",0.237911,0.315280,0.359768,0.344294,0.324952,0.357834,0.317215,0.526112,0.336557,0.444874
0,stage2,421,384,"+:113,0:103,-:168",0.268229,0.302083,0.343750,0.458333,0.453125,0.447917,0.382812,0.510417,0.455729,0.447917
0,stage3,547,258,"+:70,0:87,-:101",0.337209,0.282946,0.325581,0.457364,0.449612,0.453488,0.383721,0.418605,0.445736,0.426357
0,stage4,657,148,"+:31,0:52,-:65",0.351351,0.324324,0.344595,0.425676,0.452703,0.452703,0.236486,0.466216,0.445946,0.459459
0,stage5,725,80,"+:18,0:29,-:33",0.362500,0.262500,0.200000,0.575000,0.562500,0.537500,0.250000,0.500000,0.575000,0.512500
0,stage6,767,38,"+:6,0:16,-:16",0.421053,0.289474,0.263158,0.578947,0.578947,0.605263,0.394737,0.526316,0.578947,0.500000
0,stage7,789,16,"+:2,0:6,-:8",0.375000,0.312500,0.375000,0.500000,0.500000,0.562500,0.375000,0.562500,0.500000,0.687500


In [10]:
#xgb max_tree_depth=6
df_stage

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lrl1,lsvc,lsvcl2,rf,lrbias,xgb
0,stage0,153,652,"+:213,0:140,-:299",0.214724,0.308282,0.371166,0.523006,0.487730,0.524540,0.521472,0.429448,0.510736,0.389571
0,stage1,288,517,"+:164,0:123,-:230",0.237911,0.315280,0.359768,0.344294,0.324952,0.357834,0.317215,0.526112,0.336557,0.444874
0,stage2,421,384,"+:113,0:103,-:168",0.268229,0.302083,0.343750,0.458333,0.453125,0.447917,0.382812,0.510417,0.455729,0.447917
0,stage3,547,258,"+:70,0:87,-:101",0.337209,0.282946,0.325581,0.457364,0.449612,0.453488,0.383721,0.418605,0.445736,0.426357
0,stage4,657,148,"+:31,0:52,-:65",0.351351,0.324324,0.344595,0.425676,0.452703,0.452703,0.236486,0.466216,0.445946,0.459459
0,stage5,725,80,"+:18,0:29,-:33",0.362500,0.262500,0.200000,0.575000,0.562500,0.537500,0.250000,0.500000,0.575000,0.512500
0,stage6,767,38,"+:6,0:16,-:16",0.421053,0.289474,0.263158,0.578947,0.578947,0.605263,0.394737,0.526316,0.578947,0.500000
0,stage7,789,16,"+:2,0:6,-:8",0.375000,0.312500,0.375000,0.500000,0.500000,0.562500,0.375000,0.562500,0.500000,0.687500


### save the trained models

In [14]:
import pickle 
eventsname = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']
events = set(stagedata['eventid'])
#for eventid in events:
eventid = 1
name = 'lsvc'
signmodel = 'signmodel-' + eventsname[eventid] + '-lsvc' + '.pkl'
if True:
    print('Testset = %s'%eventsname[eventid])
    
    train, test, train_x, train_y, test_x, test_y = split_by_eventid(stagedata, eventid)

    print('[*] predict with %s model'%name)
    clf = get_classifier(name)
    clf.fit(train_x, train_y)

    pred_y = clf.predict(test_x)
    score = evaluate(test_y, pred_y)
    
    print('rec:%s'%score)
    
    #save the model
    with open(signmodel, 'wb') as fout:   
        pickle.dump((clf, test_x, test_y), fout)


Testset = Indy500
[*] predict with lsvc model
precision=[0.54716981 0.17241379 0.7027027 ], recall=[0.76315789 0.2173913  0.29545455], f1=[0.63736264 0.19230769 0.416     ], accuracy=0.52
rec:0.5244444444444445


In [15]:
#load model and predict
with open(modelname, 'rb') as fin:
    clf, test_x, test_y = pickle.load(fin)
    
EMPTY = 100
def predict(carno, stageid):
    #
    # stageid is the id of pitstop, start from 0
    #
    #find input x <eventid, car_num, stageid>
    input_x = []
    for x in test_x:
        if ((x[1] == carno) and (x[2] == stageid)):
            input_x = x.reshape((1,-1))
            #input_x = test_x[(test_x[:,1] == carno) and (test_x[:,2] == stageid)]
            pred_y = clf.predict(input_x)
            #print('prediction(car-%d,stage-%d):%s'%(carno, stageid, pred_y))
            return pred_y[0]
    else:
        return EMPTY
    
yhat = clf.predict(test_x)

#check carno 12
carno=12
idx = (test_x[:,1]==carno)
_yhat = yhat[idx]

ret_y = []
for stageid in range(10):
    Y = predict(carno, stageid)
    if Y == EMPTY:
        break
    ret_y.append(Y)
    
#predict(12, 3)
print('trueth:', test_y[idx])
print('prediction:', _yhat)
print('prediction:', ret_y)




('trueth:', array([ 1,  1, -1,  1,  1, -1]))
('prediction:', array([ 1,  1, -1,  1,  1,  0]))
('prediction:', [1, 1, -1, 1, 1, 0])


In [ ]:
input_x = []
type(input_x)